# basic analysis

In [ ]:
data_analysis['100ma'] = data_analysis['Close'].rolling(window=100,min_periods=0).mean()
# this is used to compute the average for pass 100 price (including itself)

ax1 = plt.subplot2grid((6,1),(0,0),rowspan=5,colspan=1)
ax2 = plt.subplot2grid((6,1),(5,0),rowspan=5,colspan=1,sharex=ax1)
ax1.plot(data.index,data_analysis['Close'])
ax1.plot(data.index,data_analysis['100ma'])
ax2.plot(data.index,data_analysis['Total Trade Quantity'])
plt.show()

In [ ]:
data_ohlc = data_analysis['Close'].resample('10D').ohlc()
data_volume = data_analysis['Total Trade Quantity'].resample('10D').sum()
data_ohlc.reset_index(inplace=True) # the index should be 0 1 2 ....
data_ohlc['Date'] = data_ohlc['Date'].map(mdates.date2num) # transfer datetime to timestamp

ax1 = plt.subplot2grid((6,1),(0,0),rowspan=5,colspan=1)
ax2 = plt.subplot2grid((6,1),(5,0),rowspan=5,colspan=1,sharex=ax1)
ax1.xaxis_date()
candlestick_ohlc(ax1,data_ohlc.values,width=2,colorup='g')
ax2.fill_between(data_volume.index.map(mdates.date2num),data_volume.values,0)
plt.show() ## candlestick graph

## calculation of Relative Strength index(RSI)

In [ ]:
TechIndicator = data.copy()

In [ ]:
# Relative Strength Index
# Avg(PriceUp)/(Avg(PriceUP)+Avg(PriceDown)*100
# Where: PriceUp(t)=1*(Price(t)-Price(t-1)){Price(t)- Price(t-1)>0};
#        PriceDown(t)=-1*(Price(t)-Price(t-1)){Price(t)- Price(t-1)<0};

def rsi(values):
    up = values[values>0].mean()
    down = -1*values[values<0].mean()
    return 100 * up / (up + down)

In [ ]:
# Add Momentum_1D column for all 15 stocks.
# Momentum_1D = P(t) - P(t-1)
TechIndicator['Momentum_1D'] = (TechIndicator['Close']-TechIndicator['Close'].shift(1)).fillna(0)
TechIndicator['RSI_14D'] = TechIndicator['Momentum_1D'].rolling(center=False, window=14).apply(rsi).fillna(0)
TechIndicator.tail(5)

## Calculation of Bollinger Bands

In [ ]:
def bbands(price, length=30, numsd=2):
    """ returns average, upper band, and lower band"""
    #ave = pd.stats.moments.rolling_mean(price,length)
    ave = price.rolling(window = length, center = False).mean()
    #sd = pd.stats.moments.rolling_std(price,length)
    sd = price.rolling(window = length, center = False).std()
    upband = ave + (sd*numsd)
    dnband = ave - (sd*numsd)
    return np.round(ave,3), np.round(upband,3), np.round(dnband,3)

In [ ]:
TechIndicator['BB_Middle_Band'], TechIndicator['BB_Upper_Band'], TechIndicator['BB_Lower_Band'] = bbands(TechIndicator['Close'], length=20, numsd=1)
TechIndicator['BB_Middle_Band'] = TechIndicator['BB_Middle_Band'].fillna(0)
TechIndicator['BB_Upper_Band'] = TechIndicator['BB_Upper_Band'].fillna(0)
TechIndicator['BB_Lower_Band'] = TechIndicator['BB_Lower_Band'].fillna(0)
TechIndicator.tail()

## Calculation of Aroon Oscillator

In [ ]:
def aroon(df, tf=25):
    aroonup = []
    aroondown = []
    x = tf
    while x< len(df['High']):
        aroon_up = ((df['High'][x-tf:x].tolist().index(max(df['High'][x-tf:x])))/float(tf))*100
        aroon_down = ((df['Low'][x-tf:x].tolist().index(min(df['Low'][x-tf:x])))/float(tf))*100
        aroonup.append(aroon_up)
        aroondown.append(aroon_down)
        x+=1
    return aroonup, aroondown

In [ ]:
listofzeros = [0] * 25
up, down = aroon(TechIndicator)
aroon_list = [x - y for x, y in zip(up,down)]
if len(aroon_list)==0:
    aroon_list = [0] * TechIndicator.shape[0]
    TechIndicator['Aroon_Oscillator'] = aroon_list
else:
    TechIndicator['Aroon_Oscillator'] = listofzeros+aroon_list

## Calculation of Price Volume Trend
PVT = [((CurrentClose - PreviousClose) / PreviousClose) x Volume] + PreviousPVT

In [ ]:
TechIndicator["PVT"] = (TechIndicator['Momentum_1D']/ TechIndicator['Close'].shift(1))*TechIndicator['Total Trade Quantity']
TechIndicator["PVT"] = TechIndicator["PVT"]-TechIndicator["PVT"].shift(1)
TechIndicator["PVT"] = TechIndicator["PVT"].fillna(0)

## Calculation of Acceleration Bands

# download data from yahoo 

In [ ]:
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf

tickers_list = pd.read_csv('Yahoo_Ticker_Symbols.csv')

tickers_list.drop(columns=['Category Name'],axis=1,inplace=True)

tickers = list(tickers_list.Ticker)

len(tickers)

start_time = datetime.strptime('Jan 1 2000 ', '%b %d %Y ')
start_time

for i in range(0,21):
    start = i * 5000
    end = (i+1)*5000
    Total_data_1 = pdr.get_data_yahoo(tickers=tickers[start:end],start=start_time,end=datetime.now())
    closing_price = Total_data_1['Close']
    volume = Total_data_1['Volume']
    closing_price.to_csv('closing_part'+str(i)+'.csv')
    volume.to_csv('volume_part'+str(i)+'.csv')
    print('finished '+str(i)+'loop')

# input data as dataframe through pandas from csv files
transfer dataframe index from str to datetime to filter data on Saturday and then output as pickle file to easy input in the future analysis.

In [ ]:
import glob
import pandas as pd 
path ='data' # use your path
allFiles = glob.glob(path + "/*.csv")

clo_price_data = []
vol_data = []
for file in allFiles:
    if file.startswith('data/closing'):
        data = pd.read_csv(file).set_index('Date')
        clo_price_data.append(data)
        print('Successful import closing price file '+file)
#     else:
#         data = pd.read_csv(file).set_index('Date')
#         vol_data.append(data)
#         print('Successful import volume  file'+file)
data_clo = pd.concat(clo_price_data, axis = 1, ignore_index = False,sort=False)
data_clo.index = pd.to_datetime(data_clo.index)
# data_clo.sort_index(inplace=True)
# data_clo = data_clo.iloc[2:-1,:]

clo_price_data  = []
data_clo.index = pd.to_datetime(data_clo.index)

data_clo[data_clo.index.weekday!=6].to_pickle('data/closing_price.pkl')

# data preprocessing 

In [ ]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import seaborn as sns
import matplotlib.pyplot as plt
yf.pdr_override()

In [ ]:
clo_data = pd.read_pickle('data/closing_price.pkl')

In [ ]:
clo_data.index = pd.to_datetime(clo_data.index)
clo_data.sort_index(inplace=True)  ## transfet index from str to datetime and then sort by datetime

In [ ]:
clo_data

In [ ]:
numNAs = (clo_data.isna().sum()/len(clo_data)).copy()

numNAs.hist()

stock_list =(numNAs<=0.2).copy()
stock_list = stock_list[stock_list==True]

cleaned_clo_data = clo_data[list(stock_list.index)]

In [ ]:
cleand_data = cleaned_clo_data.iloc[cleaned_clo_data.index.weekday!=6]

In [ ]:
cleand_data.to_pickle('data/closing_price.pkl')

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
clearn_numNAs = cleaned_clo_data.isna().sum(axis=1)
plt.plot(clearn_numNAs)
ax.set_xlabel('Year')
ax.set_ylabel('Num of Nan values')

In [ ]:
import missingno as msno
msno.bar(cleaned_clo_data)

In [ ]:
cleaned_clo_data

In [ ]:
tickers_list = pd.read_csv('Yahoo_Ticker_Symbols.csv')

In [ ]:
Exchange_list = tickers_list.Exchange.unique()

In [ ]:
Ticker_df = tickers_list.set_index('Ticker')

In [ ]:
Exchange_dict = {}
for name in cleaned_clo_data.columns:
    exchange = Ticker_df.loc[name].Exchange
    if exchange not in Exchange_dict.keys():
        Exchange_dict[exchange] = [name]
    else:
        Exchange_dict[exchange].append(name)

In [ ]:
xdata = list(Exchange_dict.keys())
ydata = [len(Exchange_dict[key]) for key in xdata]

In [ ]:
fig,ax = plt.subplots(figsize=(8,5))
plt.plot(xdata,ydata)

## seperate the dataframes into several dataframes by closing price and total price 
we can get volume value and closing price for all the tickers and then we try to drop sum tickers which have nan values more than 1% of total length and then fill nan value as previous price.

In [ ]:
Closed_price = Total_data['Close']
missing_percent = Closed_price.isna().sum()/len(Closed_price)
cleaned_cloing_price = Closed_price.drop(columns=missing_percent[missing_percent>0.01].index)
cleaned_cloing_price.fillna(method='ffill',inplace=True)
cleaned_cloing_price.head()

In [ ]:
columns_list = cleaned_cloing_price.columns
Volume = Total_data['Volume']
cleaned_volume = Volume[columns_list]

In [ ]:
cleaned_volume

In [ ]:
cleaned_volume.to_csv('volume_data.csv')
cleaned_cloing_price.to_csv('cloing_data.csv')

## price return 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def log_return(row):
    r = np.log(row).diff()
    return r 

In [ ]:
log_return = np.log(cleaned_cloing_price).diff()

In [ ]:
plt.plot(log_return.iloc[10])

In [ ]:
cleaned_cloing_price.head()

In [ ]:
N = 100; # number of time series
T = 200  # time length
q = N/T  # ratio'

In [ ]:
price = cleaned_cloing_price.iloc[0:T+1, 0:N]
ret = np.log(price).diff()[1:]
corr_ret = np.corrcoef(ret.T)

In [ ]:
u, v = np.linalg.eig(corr_ret)

In [ ]:
lambda_plus = (1 + np.sqrt(q))**2

In [ ]:
def compute_C_minus_C0(lambdas, v, lambda_plus):
    # lambdas is a vector
    # v is an eigenvector matrix
    C_r = np.dot(np.dot(v[:,lambdas < lambda_plus], np.diag(lambdas[lambdas < lambda_plus])), v[:,lambdas < lambda_plus].T)
    C_M = lambdas.max() * np.dot(v[:, lambdas.argmax()], v[:, lambdas.argmax()].T)
    C_0 = C_r + C_M
    C = np.dot(np.dot(v, np.diag(lambdas)), v.T)
    return C - C_0

In [ ]:
import community 
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
C_s = np.abs(compute_C_minus_C0(u, v, lambda_plus))
G = nx.Graph(C_s)
part = community.best_partition(G)
values = [part.get(node) for node in G.nodes()]

nx.draw_spring(G, cmap = plt.get_cmap('jet'), node_color = values, node_size=30, with_labels=False)


In [ ]:
G = nx.Graph(C_s)
partition  = community.best_partition(G)
size = float(len(set(partition.values())))
pos = nx.spring_layout(G)
count = 0
for com in set(partition.values()) :
    count = count + 1.
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    nx.draw_networkx_nodes(G, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))


nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()